# 会話型インターフェイス - AI21 LLM を使ったチャットボット

> *このノートブックは、SageMaker Studioの **`Data Science 3.0`** カーネルで実行してください*

このノートブックでは、Amazon Bedrock の基盤モデルを使用したチャットボットを構築します。今回のユースケースでは、Jurrasic の基盤モデルを使用してチャットボットを構築します。

## 概要

会話型インターフェースのチャットボットや仮想アシスタントは、お客様のユーザーエクスペリエンスを向上させるために使用できます。チャットボットは、自然言語処理 (NLP) と機械学習アルゴリズムを使用して、ユーザーのクエリを理解し、応答します。チャットボットは、カスタマーサービス、販売、Eコマースなど、さまざまなアプリケーションで使用でき、ユーザーに迅速かつ効率的な対応を提供できます。ウェブサイト、SNSプラットフォーム、メッセージングアプリなど、さまざまなチャネルを介してアクセスできます。

## Amazon Bedrock を使用したチャットボット

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## ユースケース

1. チャットボット (基本) - 基盤モデルを使用したゼロショットチャットボット
1. プロンプトテンプレート (Langchain) を使用したチャットボット - プロンプトテンプレートである程度のコンテキストを提供したチャットボット
1. ペルソナを持つチャットボット - 定義された役割を持つチャットボット。例: キャリアコーチと人間の対話
1. コンテキスト認識チャットボット - 外部ファイルから埋め込みを生成し、コンテキストとして渡す。

## Amazon Bedrock でチャットボットを構築するための Langchain フレームワーク
会話型インターフェースのチャットボットでは、短期的だけでなく長期的にも、以前の対話を記憶しておくことが非常に重要です。

LangChain は、2 つの形でメモリコンポーネントを提供します。第一に、LangChain は、以前のチャットメッセージを管理および操作するためのヘルパーユーティリティを提供します。これらは、使用方法に関係なく、モジュール型で有用なように設計されています。第二に、LangChain は、これらのユーティリティをチェーンに簡単に組み込む方法を提供します。

LangChain を使用すると、強力なチャットボットを簡単に構築できるようにするためのさまざまなタイプの抽象化を簡単に定義および利用することができます。

## コンテキスを持ったチャットボットの構築 - 重要ポイント

コンテキスト認識チャットボットを構築する最初のプロセスは、コンテキストのための **埋め込みを生成** することです。通常、埋め込みモデルを実行して埋め込みを生成し、ベクトルストアのようなものに保存するインジェストプロセスがあります。この例では、Titan Embeddings モデルを使用します。

![Embeddings](./images/embeddings_lang.png)

次のプロセスは、ユーザリクエストのオーケストレーション、対話、呼び出し、そして結果の返却です。

![Chatbot](./images/chatbot_lang.png)

## アーキテクチャ 
![Context Aware Chatbot](./images/context-aware-chatbot.png)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ AWS 環境の設定に応じて、以下の行のコメントを外して編集してください。 ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## チャットボット (基本 - コンテキスト無し)

#### LangChain の ConversationChain を使用して会話を開始する
チャットボットは以前の対話を記憶する必要があります。会話メモリを使用することでそれを実現できます。会話メモリを実装する方法はいくつかあります。LangChain では、それらはすべて ConversationChain の上に構築されます。

**注:** モデルの出力は非決定論的です。

In [ ]:
from langchain.chains.conversation.base import ConversationChain
from langchain_community.llms.bedrock import Bedrock
from langchain.memory.buffer import ConversationBufferMemory

ai21_llm = Bedrock(model_id="ai21.j2-ultra-v1", client=boto3_bedrock) # 使用する環境に合わせて、適切なモデルバージョンを指定してください。
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=ai21_llm, verbose=True, memory=memory
)

try:
    
    print_ww(conversation.predict(input="Hi there!"))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

#### 新しい質問

モデルは初期メッセージで応答しました。いくつか質問してみましょう。

In [ ]:
print_ww(conversation.predict(input="Give me a few tips on how to start a new garden."))

#### 質問を続ける

「庭」という言葉を出さずに質問して、モデルが以前の会話を理解できるかどうか確認してみましょう。

In [ ]:
print_ww(conversation.predict(input="Cool. Will that work with tomatoes?"))

#### 会話を終了する

In [ ]:
print_ww(conversation.predict(input="That's all, thank you!"))

## プロンプトテンプレート (Langchain) を使用したチャットボット

[PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) は、モデルのインプットの構築を支援します。LangChain は、プロンプトの構築と操作を簡単にするためのいくつかのクラスと関数を提供しています。ここではデフォルトの PromptTemplate を使用します。

In [ ]:
from langchain.memory.buffer import ConversationBufferMemory
from langchain_core.prompts.prompt import PromptTemplate

chat_history = []

# turn verbose to true to see the full logs and documents
qa= ConversationChain(
    llm=ai21_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

print(f"ChatBot:DEFAULT:PROMPT:TEMPLATE: is ={qa.prompt.template}")

In [ ]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

チャットを始めましょう。 (以下のセルを実行して、表示されたウィジェットにメッセージを入力し `Send` をクリックします。終了する場合は `q` を送信します。)

In [ ]:
chat = ChatUX(qa)
chat.start_chat()

## ペルソナを持つチャットボット

AI アシスタントはキャリアコーチの役割を演じます。ロールプレイ対話では、チャットを開始する前にユーザーメッセージを設定する必要があります。ConversationBufferMemory は、対話を自動入力するために使用されます。

In [ ]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Context:You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am career coach and give career advice")
ai21_llm = Bedrock(model_id="ai21.j2-ultra-v1", client=boto3_bedrock) 
conversation = ConversationChain(
     llm=ai21_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="What are the career options in AI?"))

##### このペルソナの専門外の質問をしてみましょう。モデルはその質問に答えず、理由を述べるべきです。

In [ ]:
conversation.verbose = False
print_ww(conversation.predict(input="How to fix my car?"))

## コンテキストを持ったチャットボット

この使用例では、チャットボットに渡されたコンテキストから質問に答えるよう求めます。csv ファイルを読み取り、Titan 埋め込みモデルを使用してベクトルを作成します。このベクトルは FAISS に保存されます。チャットボットに質問をする際に、このベクトルを渡して答えを検索します。

#### Titan 埋め込みモデル

埋め込みは、単語、フレーズ、その他の離散的なアイテムを、連続ベクトル空間内のベクトルとして表現する方法です。これにより、機械学習モデルはこれらの表現に対して数学的操作を実行し、それらの間の意味的な関係性を捉えることができます。

これは RAG の [ドキュメント検索機能](https://labelbox.com/blog/how-vector-similarity-search-works/) に使用されます。

In [ ]:
from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.prompts.prompt import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

#### ベクトルストアとしての FAISS

検索に埋め込みを使用できるようにするには、ベクトルの類似度検索を効率的に実行できるストアが必要です。このノートブックでは、インメモリストアの FAISS を使用しています。ベクトルを永続的に保存するには、pgVector、Pinecone、Weaviate、Chroma などを使用できます。

LangChain の VectorStore API は [ドキュメント](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html) で詳細を確認してください。

FAISS ベクトルストアの詳細は、こちらの [ドキュメント](https://arxiv.org/pdf/1702.08734.pdf) を参照してください。

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

s3_path = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
!aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

#### 簡単なテストの実行

LangChain が提供するラッパークラスを使用して、ベクトルデータベースストアにクエリを実行し、関連ドキュメントを返すことができます。これはすべてのデフォルト値で QA チェーンを実行するだけです実行することができます。

In [ ]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("R in SageMaker", llm=ai21_llm))

#### チャットボットアプリケーション

チャットボットには、コンテキスト管理、履歴、ベクトルストアなど、多くのものが必要です。ConversationalRetrievalChain から始めましょう。

これは、会話メモリと RetrievalQAChain を使用しています。これにより、追加の質問に使用できるチャット履歴を渡すことができます。ソース: https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html

verbose を True に設定すると、裏で起こっているすべてのことが表示されます。

In [ ]:
from langchain.memory.buffer import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

def create_prompt_template():
    _template = """{chat_history}

Answer only with the new question.
How would you ask the question considering the previous conversation: {question}
Question:"""
    CONVO_QUESTION_PROMPT = PromptTemplate.from_template(_template)
    return CONVO_QUESTION_PROMPT

memory_chain = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)
chat_history=[]

#### ConversationRetrievalChain で使用されるパラーメータ

* **retriever**: `VectorStoreRetriever` を使用しました。これは `VectorStore` によってバックエンドでサポートされています。テキストを検索するには、2 つの検索タイプ `"similarity"` または `"mmr"` を選択できます。`search_type="similarity"` は、リトリーバー内の類似度検索を使用し、質問ベクトルと最も類似したテキストチャンクベクトルを選択します。

* **memory**: 履歴を保存するためのメモリーチェーン。

* **condense_question_prompt**: ユーザーからの質問を受け取り、以前の会話とその質問を使用して、スタンドアロンの質問を作成します。

* **chain_type**: チャット履歴が長く、コンテキストに適合しない場合は、このパラメータを使用します。オプションは `stuff`、`refine`、`map_reduce`、`map-rerank` があります。

質問の範囲がコンテキストの範囲外の場合、モデルは答えが分からないと返答します。

**注:** チェーンの動作が気になる場合は、`verbose=True` の行のコメントを外してください。

In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm=ai21_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    #verbose=True,
    #condense_question_prompt=CONDENSE_QUESTION_PROMPT, # create_prompt_template(), 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=100
)

qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}:

Use at maximum 3 sentences to answer the question. 

{question}:

If the answer is not in the context say "Sorry, I don't know, as the answer was not found in the context."

Answer:""")

チャットを始めましょう。

In [ ]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

## まとめ

このラボでは、以下のパターンで会話型インターフェースを作成する方法を確認しました:
1. チャットボット(基本 - コンテキストなし)

1. プロンプトテンプレート (Langchain) を使用したチャットボット

1. ペルソナを持つチャットボット

1. コンテキストを持つチャットボット